In [ ]:
path = 'Dataset Thingy'

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Mine Master

In [ ]:
np.random.seed(42)
# ==== 1. Mine Master ====
mine_master = pd.DataFrame({
    'mine_id': [f'MINE_{i}' for i in range(1, 6)],
    'mine_name': [
        'Tambang Batubara Sungai Merah',
        'Tambang Batubara Tanjung Bara',
        'Tambang Batubara Indrapura',
        'Tambang Batubara Mahakam',
        'Tambang Batubara Sebuku'
    ],
    'location': ['Barito', 'Kutai Timur', 'Batu Bara', 'Samarinda', 'Nunukan'],
    'region': ['Kalimantan Timur', 'Kalimantan Timur', 'Sumatera Selatan', 'Kalimantan Timur', 'Kalimantan Selatan'],
    'status': ['Active'] * 5,
    'start_date': pd.date_range('2018-01-01', periods=5, freq='Y'),
    'remarks': ['Area utama produksi', 'Ekspansi', 'Cadangan batubara tinggi', 'Dekat pelabuhan', 'Daerah baru']
})

/tmp/ipython-input-3377833133.py:15: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  'start_date': pd.date_range('2018-01-01', periods=5, freq='Y'),


In [ ]:
mine_master

,mine_id,mine_name,location,region,status,start_date,remarks
0,MINE_1,Tambang Batubara Sungai Merah,Barito,Kalimantan Timur,Active,2018-12-31,Area utama produksi
1,MINE_2,Tambang Batubara Tanjung Bara,Kutai Timur,Kalimantan Timur,Active,2019-12-31,Ekspansi
2,MINE_3,Tambang Batubara Indrapura,Batu Bara,Sumatera Selatan,Active,2020-12-31,Cadangan batubara tinggi
3,MINE_4,Tambang Batubara Mahakam,Samarinda,Kalimantan Timur,Active,2021-12-31,Dekat pelabuhan
4,MINE_5,Tambang Batubara Sebuku,Nunukan,Kalimantan Selatan,Active,2022-12-31,Daerah baru


# Equipment Inventory

In [ ]:
# ==== 2. Equipment Inventory ====
weight_mine = [0.35, 0.25, 0.1, 0.15, 0.15]
equipment_types = ['Excavator', 'Dump Truck', 'Loader']

# Brand – Model per kategori
brand_model = {
    'Excavator': {
        'Caterpillar': ['320D', '336D', '349D'],
        'Komatsu': ['PC200', 'PC300', 'PC400'],
        'Hitachi': ['ZX870', 'EX200', 'EX1200'],
        'Liebherr': ['R9800']
    },
    'Dump Truck': {
        'Caterpillar': ['785', '789', '793'],
        'Komatsu': ['HD785', 'HD1500'],
        'Hitachi': ['EH3500AC-3', 'EH4000AC-3'],
        'BelAZ': ['75131', '75710'],
        'Liebherr': ['T 264', 'T 284'],
        'Volvo': ['A40G']
    },
    'Loader': {
        'Caterpillar': ['950H', '966H', '988K', '992K'],
        'Komatsu': ['WA380', 'WA470', 'WA600'],
        'Hitachi': ['ZW370', 'ZW550'],
        'Liebherr': ['L556', 'L580'],
        'Volvo': ['L150H', 'L220H']
    }
}

# memilih model sesuai kategori
def choose_model(e_type, brand):
    try:
        return np.random.choice(brand_model[e_type][brand])
    except KeyError:
        return None

# generate dataframe awal
mine_ids_for_equipment = np.random.choice(mine_master['mine_id'], 50, p=weight_mine)
equipment_inventory = pd.DataFrame({
    'equipment_id': [f'EQ{i:03d}-{mine_ids_for_equipment[i-1][5]}' for i in range(1, 51)],
    'mine_id': mine_ids_for_equipment,
    'equipment_type': np.random.choice(equipment_types, 50),
})

# pilih brand sesuai tipe alat
equipment_inventory['brand'] = equipment_inventory['equipment_type'].apply(
    lambda t: np.random.choice(list(brand_model[t].keys()))
)

# pilih model sesuai brand & tipe
equipment_inventory['model'] = equipment_inventory.apply(
    lambda row: choose_model(row['equipment_type'], row['brand']), axis=1
)

# kapasitas excavator berdasarkan model
def assign_excavator_capacity(model):
    model = str(model).lower()

    if 'r9800' in model:
        return np.random.randint(40, 46)
    elif 'ex1200' in model or 'zx870' in model:
        return np.random.randint(6, 11)
    elif 'pc200' in model or 'ex200' in model:
        return np.random.randint(1, 2)
    else:
        return np.random.randint(3, 28)

# kapasitas per tipe alat
def assign_capacity(e_type, model):
    if e_type == 'Excavator':
        return assign_excavator_capacity(model)
    elif e_type == 'Dump Truck':
        return np.random.randint(90, 250)
    elif e_type == 'Loader':
        return np.random.randint(3, 15)
    else:
        return None

equipment_inventory['base_capacity_ton'] = equipment_inventory.apply(
    lambda row: assign_capacity(row['equipment_type'], row['model']),
    axis=1
)

equipment_inventory['status'] = np.random.choice(['Active', 'Maintenance'], 50, p=[0.8, 0.2])
equipment_inventory['last_maintenance'] = pd.date_range('2025-09-01', periods=50, freq='3D')
equipment_inventory['operator_id'] = [f'OP{i:03d}' for i in range(1, 51)]

In [ ]:
equipment_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   equipment_id       50 non-null     object        
 1   mine_id            50 non-null     object        
 2   equipment_type     50 non-null     object        
 3   brand              50 non-null     object        
 4   model              50 non-null     object        
 5   base_capacity_ton  50 non-null     int64         
 6   status             50 non-null     object        
 7   last_maintenance   50 non-null     datetime64[ns]
 8   operator_id        50 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.6+ KB


In [ ]:
equipment_inventory.head()

,equipment_id,mine_id,equipment_type,brand,model,base_capacity_ton,status,last_maintenance,operator_id
0,EQ001-2,MINE_2,Loader,Volvo,L220H,14,Active,2025-09-01,OP001
1,EQ002-5,MINE_5,Dump Truck,Hitachi,EH4000AC-3,128,Maintenance,2025-09-04,OP002
2,EQ003-4,MINE_4,Dump Truck,BelAZ,75131,219,Active,2025-09-07,OP003
3,EQ004-2,MINE_2,Dump Truck,Volvo,A40G,220,Active,2025-09-10,OP004
4,EQ005-1,MINE_1,Dump Truck,Hitachi,EH4000AC-3,202,Maintenance,2025-09-13,OP005


In [ ]:
equipment_inventory.tail()

,equipment_id,mine_id,equipment_type,brand,model,base_capacity_ton,status,last_maintenance,operator_id
45,EQ046-3,MINE_3,Dump Truck,BelAZ,75710,179,Active,2026-01-14,OP046
46,EQ047-1,MINE_1,Excavator,Liebherr,R9800,42,Active,2026-01-17,OP047
47,EQ048-2,MINE_2,Loader,Liebherr,L556,5,Active,2026-01-20,OP048
48,EQ049-2,MINE_2,Loader,Liebherr,L580,6,Active,2026-01-23,OP049
49,EQ050-1,MINE_1,Dump Truck,BelAZ,75710,185,Active,2026-01-26,OP050


# Weather

In [ ]:
weather = pd.read_csv(f'{path}/weather_data.csv')

weather

,time,sp,tp,tcc,ptype,latitude,longitude,location,10u,10v,2t,2d,weather_id,humidity_pct
0,2018-12-31,100495.95,0.000717,0.982983,1.0,-1.933330,115.100000,MINE_1,0.149496,0.904919,300.17657,297.86777,W00000-1,87.217370
1,2019-01-01,100627.58,0.001328,1.000000,1.0,-1.933330,115.100000,MINE_1,0.211508,-0.164572,299.22372,297.80704,W00001-1,91.921403
2,2019-01-02,100689.15,0.000813,0.999780,1.0,-1.933330,115.100000,MINE_1,0.397208,-0.634473,299.59854,298.20108,W00002-1,92.048375
3,2019-01-03,100801.21,0.000159,0.993750,1.0,-1.933330,115.100000,MINE_1,-0.359683,0.437537,299.47665,297.63965,W00003-1,89.655613
4,2019-01-04,100706.94,0.000037,0.850818,0.6,-1.933330,115.100000,MINE_1,-1.016595,-0.486227,301.02400,298.92447,W00004-1,88.382719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8862,2025-11-07,100227.97,0.000016,0.901123,0.5,-1.933330,115.100000,MINE_1,-0.204282,0.247272,301.37656,298.78387,W08862-1,85.865957
8863,2025-11-07,99849.97,0.001444,0.980957,1.0,1.037690,117.831120,MINE_2,-0.980258,-0.834955,299.42538,298.30304,W08863-2,93.560544
8864,2025-11-07,99753.97,0.000011,0.995117,1.0,-3.711416,104.007235,MINE_3,0.124234,-0.822455,302.88397,296.60223,W08864-3,69.061801
8865,2025-11-07,100446.97,0.000003,0.988632,0.0,-0.502106,117.153709,MINE_4,-1.063461,-0.569916,300.94376,298.20303,W08865-4,85.070436


In [ ]:
wind_speed = np.sqrt(weather['10u']**2 + weather['10v']**2) #dalam m/s
wind_speed_kmh = wind_speed *(1e3/(60*60))

weather['wind_speed_kmh'] = wind_speed_kmh

In [ ]:
tp_mm = weather['tp']*1000
weather['tp_mm'] = tp_mm

In [ ]:
temp_c = weather['2t'] - 273.15
weather['temp_c'] = temp_c

In [ ]:
def classify_weather(row):
    rain = row['tp_mm']
    hum  = row['humidity_pct']
    wind = row['wind_speed_kmh']
    tcc  = row['tcc']  # cloud cover 0–1

    # --- 1. Tidak hujan ---
    if 0 < rain < 1.00:

        # cerah
        if tcc < 0.3 and hum < 70:
            return "Cerah"

        # berawan
        if 0.3 <= tcc < 0.7:
            return "Berawan"

        # mendung
        if tcc >= 0.7 or hum >= 80:
            return "Mendung"

        return "Cerah"   # fallback

    # --- 2. Hujan ringan ---
    if 1.00 < rain <= 2.4:
        return "Hujan ringan"

    # --- 3. Hujan sedang ---
    if 2.4 < rain <= 15:
        return "Hujan sedang"

    # --- 5. Hujan lebat ---
    return "Hujan lebat"

In [ ]:
remarks = weather.apply(classify_weather, axis=1)
remarks.unique()

array(['Mendung', 'Hujan ringan', 'Berawan', 'Cerah', 'Hujan lebat',
       'Hujan sedang'], dtype=object)

In [ ]:
remarks.value_counts()

,count
Mendung,6099
Berawan,1384
Hujan ringan,795
Hujan lebat,432
Cerah,99
Hujan sedang,58


In [ ]:
weather['remarks'] = remarks

In [ ]:
weather

,time,sp,tp,tcc,ptype,latitude,longitude,location,10u,10v,2t,2d,weather_id,humidity_pct,wind_speed_kmh,tp_mm,temp_c,remarks
0,2018-12-31,100495.95,0.000717,0.982983,1.0,-1.933330,115.100000,MINE_1,0.149496,0.904919,300.17657,297.86777,W00000-1,87.217370,0.254774,0.716591,27.02657,Mendung
1,2019-01-01,100627.58,0.001328,1.000000,1.0,-1.933330,115.100000,MINE_1,0.211508,-0.164572,299.22372,297.80704,W00001-1,91.921403,0.074442,1.328278,26.07372,Hujan ringan
2,2019-01-02,100689.15,0.000813,0.999780,1.0,-1.933330,115.100000,MINE_1,0.397208,-0.634473,299.59854,298.20108,W00002-1,92.048375,0.207931,0.813389,26.44854,Mendung
3,2019-01-03,100801.21,0.000159,0.993750,1.0,-1.933330,115.100000,MINE_1,-0.359683,0.437537,299.47665,297.63965,W00003-1,89.655613,0.157334,0.158691,26.32665,Mendung
4,2019-01-04,100706.94,0.000037,0.850818,0.6,-1.933330,115.100000,MINE_1,-1.016595,-0.486227,301.02400,298.92447,W00004-1,88.382719,0.313025,0.036812,27.87400,Mendung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8862,2025-11-07,100227.97,0.000016,0.901123,0.5,-1.933330,115.100000,MINE_1,-0.204282,0.247272,301.37656,298.78387,W08862-1,85.865957,0.089095,0.015736,28.22656,Mendung
8863,2025-11-07,99849.97,0.001444,0.980957,1.0,1.037690,117.831120,MINE_2,-0.980258,-0.834955,299.42538,298.30304,W08863-2,93.560544,0.357682,1.443625,26.27538,Hujan ringan
8864,2025-11-07,99753.97,0.000011,0.995117,1.0,-3.711416,104.007235,MINE_3,0.124234,-0.822455,302.88397,296.60223,W08864-3,69.061801,0.231051,0.010729,29.73397,Mendung
8865,2025-11-07,100446.97,0.000003,0.988632,0.0,-0.502106,117.153709,MINE_4,-1.063461,-0.569916,300.94376,298.20303,W08865-4,85.070436,0.335152,0.002861,27.79376,Mendung


In [ ]:
weather_data = weather.loc[:, ['weather_id', 'location', 'time', 'tp_mm', 'temp_c', 'wind_speed_kmh', 'humidity_pct', 'remarks']]
weather_data.columns = ['weather_id', 'mine_id', 'date', 'rainfall_mm', 'temperature_c', 'wind_speed_kmh', 'humidity_pct', 'remark']
weather_data

,weather_id,mine_id,date,rainfall_mm,temperature_c,wind_speed_kmh,humidity_pct,remark
0,W00000-1,MINE_1,2018-12-31,0.716591,27.02657,0.254774,87.217370,Mendung
1,W00001-1,MINE_1,2019-01-01,1.328278,26.07372,0.074442,91.921403,Hujan ringan
2,W00002-1,MINE_1,2019-01-02,0.813389,26.44854,0.207931,92.048375,Mendung
3,W00003-1,MINE_1,2019-01-03,0.158691,26.32665,0.157334,89.655613,Mendung
4,W00004-1,MINE_1,2019-01-04,0.036812,27.87400,0.313025,88.382719,Mendung
...,...,...,...,...,...,...,...,...
8862,W08862-1,MINE_1,2025-11-07,0.015736,28.22656,0.089095,85.865957,Mendung
8863,W08863-2,MINE_2,2025-11-07,1.443625,26.27538,0.357682,93.560544,Hujan ringan
8864,W08864-3,MINE_3,2025-11-07,0.010729,29.73397,0.231051,69.061801,Mendung
8865,W08865-4,MINE_4,2025-11-07,0.002861,27.79376,0.335152,85.070436,Mendung


In [ ]:
weather_data.tail(10)

,weather_id,mine_id,date,rainfall_mm,temperature_c,wind_speed_kmh,humidity_pct,remark
8857,W08857-1,MINE_1,2025-11-06,0.096798,26.23177,0.305709,89.792848,Mendung
8858,W08858-2,MINE_2,2025-11-06,0.073147,27.19740,0.461283,86.195735,Berawan
8859,W08859-3,MINE_3,2025-11-06,0.005913,30.17007,0.285171,63.464014,Mendung
8860,W08860-4,MINE_4,2025-11-06,0.362015,27.30563,0.409604,88.980745,Mendung
8861,W08861-5,MINE_5,2025-11-06,0.040340,28.37515,0.804649,81.040949,Mendung
8862,W08862-1,MINE_1,2025-11-07,0.015736,28.22656,0.089095,85.865957,Mendung
8863,W08863-2,MINE_2,2025-11-07,1.443625,26.27538,0.357682,93.560544,Hujan ringan
8864,W08864-3,MINE_3,2025-11-07,0.010729,29.73397,0.231051,69.061801,Mendung
8865,W08865-4,MINE_4,2025-11-07,0.002861,27.79376,0.335152,85.070436,Mendung
8866,W08866-5,MINE_5,2025-11-07,0.083446,28.70626,0.398535,79.933292,Mendung



```
# ==== 3. Weather Data ====
dates = pd.date_range('2025-10-01', '2025-10-31')
weather_data = pd.DataFrame({
    'weather_id': [f'W{i:04d}' for i in range(len(dates) * 5)],
    'mine_id': np.repeat(mine_master['mine_id'], len(dates)),
    'date': list(dates) * 5,
    'rainfall_mm': np.random.gamma(2, 10, len(dates)*5).round(1),
    'temperature_c': np.random.normal(30, 2, len(dates)*5).round(1),
    'wind_speed_kmh': np.random.uniform(5, 20, len(dates)*5).round(1),
    'humidity_pct': np.random.randint(60, 95, len(dates)*5),
    'remark': np.random.choice(['Cerah', 'Hujan ringan', 'Hujan sedang', 'Hujan lebat'], len(dates)*5)
})
```



# Road Condition

In [ ]:
# ==== 4. Road Condition (Improved, more realistic) ====

all_segments_data = []
road_id_counter = 1

for mine_id in mine_master['mine_id']:
    num_segments_for_mine = np.random.randint(4, 6) # 4 or 5 segments

    for i in range(1, num_segments_for_mine + 1):
        segment_name = f'Segment {i}'

        condition_level = np.random.choice(['Good', 'Fair', 'Poor'], p=[0.5, 0.3, 0.2])
        accessibility_pct = np.random.randint(60, 100)

        # Generate a random date for last_inspection within a reasonable range
        start_date = pd.to_datetime('2025-10-01')
        end_date = pd.to_datetime('2025-10-31')
        random_days = np.random.randint((end_date - start_date).days + 1)
        last_inspection = start_date + pd.Timedelta(days=random_days)

        # Determine remarks based on real-life logic
        remark = ''
        if condition_level == 'Good' and accessibility_pct >= 85:
            remark = 'Lancar'
        elif condition_level == 'Fair' or (70 <= accessibility_pct < 85):
            remark = 'Perlu perbaikan'
        elif condition_level == 'Poor':
            if accessibility_pct < 70:
                remark = 'Drainase buruk'
            else:
                remark = 'Lumpur'
        else:
            remark = 'Lancar' # fallback

        all_segments_data.append({
            'road_id': f'RD{road_id_counter:03d}-{mine_id[-1:]}',
            'mine_id': mine_id,
            'segment_name': segment_name,
            'condition_level': condition_level,
            'accessibility_pct': accessibility_pct,
            'last_inspection': last_inspection,
            'remark': remark
        })
        road_id_counter += 1

road_condition = pd.DataFrame(all_segments_data)

In [ ]:
road_condition

,road_id,mine_id,segment_name,condition_level,accessibility_pct,last_inspection,remark
0,RD001-1,MINE_1,Segment 1,Good,97,2025-10-06,Lancar
1,RD002-1,MINE_1,Segment 2,Good,67,2025-10-27,Lancar
2,RD003-1,MINE_1,Segment 3,Good,80,2025-10-30,Perlu perbaikan
3,RD004-1,MINE_1,Segment 4,Fair,92,2025-10-05,Perlu perbaikan
4,RD005-1,MINE_1,Segment 5,Good,78,2025-10-04,Perlu perbaikan
5,RD006-2,MINE_2,Segment 1,Fair,76,2025-10-12,Perlu perbaikan
6,RD007-2,MINE_2,Segment 2,Poor,88,2025-10-14,Lumpur
7,RD008-2,MINE_2,Segment 3,Poor,94,2025-10-28,Lumpur
8,RD009-2,MINE_2,Segment 4,Good,83,2025-10-29,Perlu perbaikan
9,RD010-3,MINE_3,Segment 1,Fair,90,2025-10-03,Perlu perbaikan


# Production Plan

## Target

In [ ]:
# Cari tahun mulai tiap tambang
mine_master['start_year'] = mine_master['start_date'].dt.year

all_years = list(range(2019, 2026))  # data target 2019–2025

annual_records = []

for _, row in mine_master.iterrows():
    for yr in all_years:
        # hanya buat target untuk tahun >= tahun mulai operasi
        if yr >= row['start_year']+1:

            # target baseline berbeda untuk tiap tambang (5–8 juta ton)
            base = np.random.randint(5_000_000, 8_000_000)

            # tambahkan variasi ±10%
            target = int(base * np.random.uniform(0.9, 1.1))

            annual_records.append({
                'mine_id': row['mine_id'],
                'year': yr,
                'annual_target_ton': target
            })

annual_target = pd.DataFrame(annual_records)

In [ ]:
annual_target.head()

,mine_id,year,annual_target_ton
0,MINE_1,2019,4934180
1,MINE_1,2020,5922377
2,MINE_1,2021,5124843
3,MINE_1,2022,8554125
4,MINE_1,2023,5380828


In [ ]:
# Monthly target
# Pola musiman realistis (total 1.0)
seasonal_weights = {
    1: 0.06, 2: 0.07, 3: 0.08, 4: 0.08,
    5: 0.09, 6: 0.10, 7: 0.11, 8: 0.11,
    9: 0.10, 10: 0.10, 11: 0.06, 12: 0.04
}

monthly_records = []

for _, row in annual_target.iterrows():
    annual = row['annual_target_ton']

    for month in range(1, 13):
        monthly_target = int(annual * seasonal_weights[month])

        monthly_records.append({
            'mine_id': row['mine_id'],
            'year': row['year'],
            'month': month,
            'monthly_target_ton': monthly_target
        })

monthly_target = pd.DataFrame(monthly_records)

In [ ]:
monthly_target.head()

,mine_id,year,month,monthly_target_ton
0,MINE_1,2019,1,296050
1,MINE_1,2019,2,345392
2,MINE_1,2019,3,394734
3,MINE_1,2019,4,394734
4,MINE_1,2019,5,444076


In [ ]:
## Weekly Production plan
weekly_records = []

# distribusi mingguan dalam 1 bulan
weekly_ratio = [0.22, 0.26, 0.26, 0.26]

for _, row in monthly_target.iterrows():
    month_target = row['monthly_target_ton']
    year = row['year']
    month = row['month']
    mine = row['mine_id']

    # Tentukan 4 minggu start date
    month_start = pd.Timestamp(year=year, month=month, day=1)

    # Ambil 4 minggu berdasarkan kalender aktual
    weeks = [month_start + pd.Timedelta(days=7 * i) for i in range(4)]

    # generate weekly target
    for i, wk_start in enumerate(weeks):
        weekly_target = int(month_target * weekly_ratio[i])

        weekly_records.append({
            'mine_id': mine,
            'week_start': wk_start,
            'year': year,
            'month': month,
            'week_number_in_month': i + 1,
            'weekly_target_ton': weekly_target
        })

weekly_production_target = pd.DataFrame(weekly_records)

In [ ]:
weekly_production_target.head(10)

,mine_id,week_start,year,month,week_number_in_month,weekly_target_ton
0,MINE_1,2019-01-01,2019,1,1,65131
1,MINE_1,2019-01-08,2019,1,2,76973
2,MINE_1,2019-01-15,2019,1,3,76973
3,MINE_1,2019-01-22,2019,1,4,76973
4,MINE_1,2019-02-01,2019,2,1,75986
5,MINE_1,2019-02-08,2019,2,2,89801
6,MINE_1,2019-02-15,2019,2,3,89801
7,MINE_1,2019-02-22,2019,2,4,89801
8,MINE_1,2019-03-01,2019,3,1,86841
9,MINE_1,2019-03-08,2019,3,2,102630


## Production plan

In [ ]:
# --- Prepare weather data once for efficiency ---
weather_daily = weather_data.copy()
weather_daily['date'] = pd.to_datetime(weather_daily['date'])

# --- Aggregate weather data based on weekly_production_target's week_start ---
unique_production_weeks = weekly_production_target[['mine_id', 'week_start']].drop_duplicates().reset_index(drop=True)

production_week_weather_records = []

for _, pw_row in unique_production_weeks.iterrows():
    m_id = pw_row['mine_id']
    week_start_date = pw_row['week_start']
    week_end_date = week_start_date + pd.Timedelta(days=6) # A full 7-day week (inclusive)

    # Filter weather_daily for the current mine and week
    weekly_weather_data_for_mine = weather_daily[
        (weather_daily['mine_id'] == m_id) &
        (weather_daily['date'] >= week_start_date) &
        (weather_daily['date'] <= week_end_date)
    ]

    # Aggregate remarks and rainfall for this 7-day period
    if not weekly_weather_data_for_mine.empty:
        # Get the mode (most frequent) weather remark
        # Using .iloc[0] to handle cases where mode returns multiple values (takes the first one)
        weather_remark = weekly_weather_data_for_mine['remark'].mode().iloc[0]
        # Sum rainfall for the week
        rainfall_mm_sum = weekly_weather_data_for_mine['rainfall_mm'].sum()
    else:
        # Default values if no weather data is available for the week
        # This might happen if weather_data doesn't cover all production weeks
        weather_remark = 'Mendung' # Sensible default
        rainfall_mm_sum = 0.0      # No rain if no data

    production_week_weather_records.append({
        'mine_id': m_id,
        'week_start': week_start_date,
        'remark': weather_remark,
        'rainfall_mm_agg': rainfall_mm_sum
    })

weather_agg_for_production_weeks = pd.DataFrame(production_week_weather_records)


In [ ]:
# ==== 5. Production Plan (REALISTIC PER MINE, MERGED, COLUMN NAMES UNCHANGED) ====
# Define factors (moving these up as they are used multiple times)
weather_factor = {
    'Cerah': 1.00,
    'Berawan': 0.99,
    'Mendung': 0.97,
    'Hujan ringan': 0.94,
    'Hujan sedang': 0.88,
    'Hujan lebat': 0.75
}
road_factor = {'Good': 1.00, 'Fair': 0.92, 'Poor': 0.78}

production_plan_list = []
for idx, row in mine_master.iterrows():
    m_id = row['mine_id']

    # Get weekly targets for the current mine
    mine_weekly_targets = weekly_production_target[
        weekly_production_target['mine_id'] == m_id
    ].copy()

    num_weekly_entries = len(mine_weekly_targets)

    # Buat plan_id yang mengandung kode mine
    plan_ids = [f"PLAN{i:04d}-{m_id[-1:]}" for i in range(1, num_weekly_entries + 1)]

    planned = mine_weekly_targets['weekly_target_ton'].values

    # --- Get weather for each weekly entry --
    current_mine_weather_info = pd.merge(
        mine_weekly_targets[['mine_id', 'week_start']], # Only relevant keys for merge
        weather_agg_for_production_weeks,
        on=['mine_id', 'week_start'],
        how='left' # Ensure all target weeks are kept
    )

    # Handle cases where a specific week might not have weather data from the aggregation
    current_mine_weather_info['remark'] = current_mine_weather_info['remark'].fillna('Mendung')

    f_weather = np.array([weather_factor[w] for w in current_mine_weather_info['remark'].values])

    # --- Generate road conditions and availability for each weekly entry ---
    road = np.random.choice(['Good', 'Fair', 'Poor'], num_weekly_entries,
                             p=[0.50, 0.35, 0.15])
    f_road = np.array([road_factor[r] for r in road])

    availability = np.random.randint(70, 100, num_weekly_entries) / 100

    # Actual realistic calculation
    actual = planned * f_weather * f_road * availability

    variance = ((actual - planned) / planned * 100).round(2)
    status = np.where(variance >= -5, 'On Track', 'Below Target')

    df = pd.DataFrame({
        'plan_id': plan_ids,
        'mine_id': [m_id] * num_weekly_entries,
        'week_start': mine_weekly_targets['week_start'].values,
        'planned_output_ton': planned,
        'actual_output_ton': actual.round(0),
        'target_variance_pct': variance,
        'status': status,
        'updated_by': np.random.choice(['MPLNR01', 'MPLNR02', 'MPLNR03'], num_weekly_entries)
    })

    production_plan_list.append(df)

# === GABUNGKAN HASIL 5 MINE JADI SATU DATASET ===
production_plan = pd.concat(production_plan_list, ignore_index=True)

In [ ]:
production_plan = production_plan.sort_values('week_start')

In [ ]:
production_plan['status'].value_counts()

,count
status,
Below Target,1157
On Track,43


In [ ]:
production_plan.head(10)

,plan_id,mine_id,week_start,planned_output_ton,actual_output_ton,target_variance_pct,status,updated_by
0,PLAN0001-1,MINE_1,2019-01-01,65131,47661.0,-26.82,Below Target,MPLNR02
1,PLAN0002-1,MINE_1,2019-01-08,76973,51518.0,-33.07,Below Target,MPLNR02
2,PLAN0003-1,MINE_1,2019-01-15,76973,50831.0,-33.96,Below Target,MPLNR02
3,PLAN0004-1,MINE_1,2019-01-22,76973,70184.0,-8.82,Below Target,MPLNR01
4,PLAN0005-1,MINE_1,2019-02-01,75986,51594.0,-32.10,Below Target,MPLNR01
5,PLAN0006-1,MINE_1,2019-02-08,89801,78396.0,-12.70,Below Target,MPLNR03
6,PLAN0007-1,MINE_1,2019-02-15,89801,72926.0,-18.79,Below Target,MPLNR01
7,PLAN0008-1,MINE_1,2019-02-22,89801,78396.0,-12.70,Below Target,MPLNR01
8,PLAN0009-1,MINE_1,2019-03-01,86841,63177.0,-27.25,Below Target,MPLNR01
9,PLAN0010-1,MINE_1,2019-03-08,102630,74663.0,-27.25,Below Target,MPLNR02


In [ ]:
production_plan.tail(10)

,plan_id,mine_id,week_start,planned_output_ton,actual_output_ton,target_variance_pct,status,updated_by
1054,PLAN0191-4,MINE_4,2025-12-15,59578,53167.0,-10.76,Below Target,MPLNR03
1198,PLAN0143-5,MINE_5,2025-12-15,84764,76466.0,-9.79,Below Target,MPLNR02
622,PLAN0287-2,MINE_2,2025-12-15,81761,64240.0,-21.43,Below Target,MPLNR01
862,PLAN0239-3,MINE_3,2025-12-15,77721,59558.0,-23.37,Below Target,MPLNR02
334,PLAN0335-1,MINE_1,2025-12-15,64325,56256.0,-12.54,Below Target,MPLNR02
335,PLAN0336-1,MINE_1,2025-12-22,64325,49916.0,-22.40,Below Target,MPLNR03
1055,PLAN0192-4,MINE_4,2025-12-22,59578,49122.0,-17.55,Below Target,MPLNR03
623,PLAN0288-2,MINE_2,2025-12-22,81761,54723.0,-33.07,Below Target,MPLNR02
863,PLAN0240-3,MINE_3,2025-12-22,77721,67850.0,-12.70,Below Target,MPLNR01
1199,PLAN0144-5,MINE_5,2025-12-22,84764,52589.0,-37.96,Below Target,MPLNR03


In [ ]:
production_plan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   plan_id              1200 non-null   object        
 1   mine_id              1200 non-null   object        
 2   week_start           1200 non-null   datetime64[ns]
 3   planned_output_ton   1200 non-null   int64         
 4   actual_output_ton    1200 non-null   float64       
 5   target_variance_pct  1200 non-null   float64       
 6   status               1200 non-null   object        
 7   updated_by           1200 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 84.4+ KB


# Effective Capacity

In [ ]:
weather_agg_for_production_weeks

,mine_id,week_start,remark,rainfall_mm_agg
0,MINE_1,2019-01-01,Mendung,4.494286
1,MINE_1,2019-01-08,Mendung,2.023125
2,MINE_1,2019-01-15,Mendung,2.203274
3,MINE_1,2019-01-22,Mendung,3.312397
4,MINE_1,2019-02-01,Mendung,3.595924
...,...,...,...,...
1195,MINE_5,2025-11-22,Mendung,0.000000
1196,MINE_5,2025-12-01,Mendung,0.000000
1197,MINE_5,2025-12-08,Mendung,0.000000
1198,MINE_5,2025-12-15,Mendung,0.000000


In [ ]:
# ==============================
# ==== EFFECTIVE CAPACITY ======
# ==============================

production_weather = production_plan.merge(weather_agg_for_production_weeks, on=['mine_id', 'week_start'], how='left')
effective_rows = []

# Define factors here for self-containment, as they were in a previous cell's loop
weather_factor = {
    'Cerah': 1.00,
    'Berawan': 0.99,
    'Mendung': 0.97,
    'Hujan ringan': 0.94,
    'Hujan sedang': 0.88,
    'Hujan lebat': 0.75
}
road_factor = {'Good': 1.00, 'Fair': 0.92, 'Poor': 0.78}


for idx, row in production_weather.iterrows(): # Iterate over production_plan

    mine = row['mine_id']
    plan_id = row['plan_id']
    week = row['week_start']

    # ===== Map Weather =====
    weather_cond = row['remark']
    rain = row['rainfall_mm_agg']

    # ===== Road condition logis berdasarkan hujan =====
    # This logic seems reasonable based on rainfall data, assuming rainfall is sum for the week.
    if rain > 40:
        road_cond = 'Poor'
    elif rain > 20:
        road_cond = 'Fair'
    else:
        road_cond = np.random.choice(['Good', 'Fair'], p=[0.7, 0.3])

    # ===== Pick equipment dari mine =====
    eq_df = equipment_inventory[equipment_inventory['mine_id'] == mine]
    if len(eq_df) == 0:
        # Fallback if no equipment for mine
        eq = equipment_inventory.sample(1).iloc[0]
    else:
        eq = eq_df.sample(1).iloc[0] # Randomly pick one equipment from the mine

    eq_id = eq['equipment_id']
    eq_type = eq['equipment_type'] # Get equipment type for the 'resource' column
    base_cap = eq['base_capacity_ton']

    # ===== Generate availability =====
    availability = np.random.randint(70, 100) # Percentage

    # ===== Effective capacity calculation =====
    eff = (
        base_cap * np.random.uniform(6, 10) # Assuming this is a multiplier for operational hours/efficiency
        * weather_factor[weather_cond]
        * road_factor[road_cond]
        * (availability / 100)
    )

    # ===== Remarks =====
    # Adjust 'remark' logic to match actual weather_factor keys
    note = (
        'Severe Weather Impact' if weather_cond in ['Hujan lebat', 'Hujan sedang'] else
        'Moderate Weather Impact' if weather_cond in ['Hujan ringan'] else
        'Road Impact' if road_cond == 'Poor' else
        'Optimal'
    )

    effective_rows.append({
        'effcap_id': f"EFC{idx:05d}-{mine[-1]}",
        'plan_id': plan_id,
        'mine_id': mine,
        'equipment_id': eq_id,
        'equipment_type': eq_type,
        'week_start': week,
        'road_condition': road_cond,
        'weather_condition': weather_cond,
        'availability_pct': availability,
        'effective_capacity_ton_day': eff.round(2),
        'remark': note
    })

effective_capacity = pd.DataFrame(effective_rows)


In [ ]:
effective_capacity

,effcap_id,plan_id,mine_id,equipment_id,equipment_type,week_start,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,remark
0,EFC00000-1,PLAN0001-1,MINE_1,EQ037-1,Excavator,2019-01-01,Fair,Mendung,73,29.46,Optimal
1,EFC00001-1,PLAN0002-1,MINE_1,EQ037-1,Excavator,2019-01-08,Good,Mendung,96,44.30,Optimal
2,EFC00002-1,PLAN0003-1,MINE_1,EQ033-1,Loader,2019-01-15,Good,Mendung,93,35.15,Optimal
3,EFC00003-1,PLAN0004-1,MINE_1,EQ007-1,Dump Truck,2019-01-22,Fair,Mendung,93,1531.46,Optimal
4,EFC00004-1,PLAN0005-1,MINE_1,EQ017-1,Loader,2019-02-01,Good,Mendung,87,41.20,Optimal
...,...,...,...,...,...,...,...,...,...,...,...
1195,EFC01195-1,PLAN0336-1,MINE_1,EQ027-1,Excavator,2025-12-22,Good,Mendung,98,399.23,Optimal
1196,EFC01196-4,PLAN0192-4,MINE_4,EQ026-4,Excavator,2025-12-22,Good,Mendung,87,218.14,Optimal
1197,EFC01197-2,PLAN0288-2,MINE_2,EQ049-2,Loader,2025-12-22,Good,Mendung,71,26.07,Optimal
1198,EFC01198-3,PLAN0240-3,MINE_3,EQ039-3,Excavator,2025-12-22,Fair,Mendung,86,50.64,Optimal


## Effective Capacity ver 2
ini per equipment per site

In [ ]:
# ==============================
# ==== NEW EFFECTIVE CAPACITY ===
# ==============================

production_weather = production_plan.merge(
    weather_agg_for_production_weeks,
    on=['mine_id', 'week_start'],
    how='left'
)

effective_rows = []

weather_factor = {
    'Cerah': 1.00,
    'Berawan': 0.99,
    'Mendung': 0.97,
    'Hujan ringan': 0.94,
    'Hujan sedang': 0.88,
    'Hujan lebat': 0.75
}

road_factor = {'Good': 1.00, 'Fair': 0.92, 'Poor': 0.78}

for _, plan_row in production_weather.iterrows():

    mine = plan_row['mine_id']
    plan_id = plan_row['plan_id']
    week = plan_row['week_start']
    weather_cond = plan_row['remark']
    rain = plan_row['rainfall_mm_agg']

    # Determine road condition from rainfall
    if rain > 50:
        road_cond = 'Poor'
    elif rain > 25:
        road_cond = 'Fair'
    else:
        road_cond = np.random.choice(['Good', 'Fair'], p=[0.75, 0.25])

    # Retrieve ALL equipment in the mine
    eq_list = equipment_inventory[equipment_inventory['mine_id'] == mine]

    # If no equipment (should never happen), fallback random
    if eq_list.empty:
        eq_list = equipment_inventory.sample(3)

    for _, eq in eq_list.iterrows():

        eq_id = eq['equipment_id']
        eq_type = eq['equipment_type']
        base_cap = eq['base_capacity_ton']

        # Availability depends on equipment status
        if eq['status'] == 'Active':
            availability = np.random.randint(75, 100)
        else:
            availability = np.random.randint(40, 70)

        if plan_row['planned_output_ton'] > 55000:
            availability = np.random.randint(60, 90)   # sedikit lebih rendah karena kejar target
        else:
            availability = np.random.randint(70, 100)

        # Calculate effective capacity
        operating_hours = np.random.uniform(6.5, 10.0)
        eff_value = (
            base_cap * operating_hours *
            weather_factor.get(weather_cond, 0.95) *
            road_factor[road_cond] *
            (availability / 100)
        )

        if plan_row['status'] == 'Delayed':
            eff_value = eff_value * np.random.uniform(0.70, 0.90)   # penalty

        # Remark logic
        if weather_cond in ['Hujan lebat', 'Hujan sedang']:
            note = 'Severe Weather Impact'
        elif weather_cond in ['Hujan ringan']:
            note = 'Moderate Weather Impact'
        elif road_cond == 'Poor':
            note = 'Road Impact'
        else:
            note = 'Optimal'


        effective_rows.append({
            'effcap_id': f"EFC-{plan_id}-{eq_id}",
            'plan_id': plan_id,
            'mine_id': mine,
            'equipment_id': eq_id,
            'equipment_type': eq_type,
            'week_start': week,
            'road_condition': road_cond,
            'weather_condition': weather_cond,
            'availability_pct': availability,
            'effective_capacity_ton_day': round(eff_value, 2),
            'remark': note
        })

effective_capacity_2 = pd.DataFrame(effective_rows)

In [ ]:
effective_capacity_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13872 entries, 0 to 13871
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   effcap_id                   13872 non-null  object        
 1   plan_id                     13872 non-null  object        
 2   mine_id                     13872 non-null  object        
 3   equipment_id                13872 non-null  object        
 4   equipment_type              13872 non-null  object        
 5   week_start                  13872 non-null  datetime64[ns]
 6   road_condition              13872 non-null  object        
 7   weather_condition           13872 non-null  object        
 8   availability_pct            13872 non-null  int64         
 9   effective_capacity_ton_day  13872 non-null  float64       
 10  remark                      13872 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)


In [ ]:
effective_capacity_2.head(30)

,effcap_id,plan_id,mine_id,equipment_id,equipment_type,week_start,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,remark
0,EFC-PLAN0001-1-EQ005-1,PLAN0001-1,MINE_1,EQ005-1,Dump Truck,2019-01-01,Good,Mendung,67,939.17,Optimal
1,EFC-PLAN0001-1-EQ006-1,PLAN0001-1,MINE_1,EQ006-1,Dump Truck,2019-01-01,Good,Mendung,65,870.51,Optimal
2,EFC-PLAN0001-1-EQ007-1,PLAN0001-1,MINE_1,EQ007-1,Dump Truck,2019-01-01,Good,Mendung,86,1517.39,Optimal
3,EFC-PLAN0001-1-EQ011-1,PLAN0001-1,MINE_1,EQ011-1,Loader,2019-01-01,Good,Mendung,84,29.92,Optimal
4,EFC-PLAN0001-1-EQ014-1,PLAN0001-1,MINE_1,EQ014-1,Excavator,2019-01-01,Good,Mendung,75,6.64,Optimal
5,EFC-PLAN0001-1-EQ015-1,PLAN0001-1,MINE_1,EQ015-1,Excavator,2019-01-01,Good,Mendung,77,72.13,Optimal
6,EFC-PLAN0001-1-EQ016-1,PLAN0001-1,MINE_1,EQ016-1,Dump Truck,2019-01-01,Good,Mendung,62,981.61,Optimal
7,EFC-PLAN0001-1-EQ017-1,PLAN0001-1,MINE_1,EQ017-1,Loader,2019-01-01,Good,Mendung,67,31.43,Optimal
8,EFC-PLAN0001-1-EQ020-1,PLAN0001-1,MINE_1,EQ020-1,Excavator,2019-01-01,Good,Mendung,66,4.22,Optimal
9,EFC-PLAN0001-1-EQ022-1,PLAN0001-1,MINE_1,EQ022-1,Excavator,2019-01-01,Good,Mendung,70,41.44,Optimal


In [ ]:
effective_capacity_2.tail()

,effcap_id,plan_id,mine_id,equipment_id,equipment_type,week_start,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,remark
13867,EFC-PLAN0144-5-EQ008-5,PLAN0144-5,MINE_5,EQ008-5,Loader,2025-12-22,Fair,Mendung,87,64.30,Optimal
13868,EFC-PLAN0144-5-EQ012-5,PLAN0144-5,MINE_5,EQ012-5,Excavator,2025-12-22,Fair,Mendung,74,26.15,Optimal
13869,EFC-PLAN0144-5-EQ034-5,PLAN0144-5,MINE_5,EQ034-5,Excavator,2025-12-22,Fair,Mendung,84,240.38,Optimal
13870,EFC-PLAN0144-5-EQ035-5,PLAN0144-5,MINE_5,EQ035-5,Loader,2025-12-22,Fair,Mendung,76,37.30,Optimal
13871,EFC-PLAN0144-5-EQ044-5,PLAN0144-5,MINE_5,EQ044-5,Loader,2025-12-22,Fair,Mendung,75,25.99,Optimal


# Production Constrain

In [ ]:
# ================================
# ==== FIXED PRODUCTION CONSTRAINTS
# ================================

np.random.seed(42)

constraint_types = ['Excavator Capacity', 'Hauling Capacity', 'Crusher Capacity']
production_list = []

# Loop based on effective_capacity
for idx, row in effective_capacity.iterrows():

    mine = row['mine_id']
    week = row['week_start']
    eq = row['equipment_id']
    # Use 'effective_capacity_ton_day' directly from effective_capacity
    base_cap = row['effective_capacity_ton_day']

    # --- Issue probability ---
    issue_type = np.random.choice(
        ['None', 'Road', 'Weather', 'Maintenance'],
        p=[0.45, 0.20, 0.25, 0.10]
    )

    # --- Apply logic ---
    if issue_type == 'Road':
        ctype = 'Hauling Capacity'
        reduction_factor = np.random.uniform(0.40, 0.75)
        remark = np.random.choice(['Jalan licin', 'Kemacetan hauling', 'Akses terhambat'])

    elif issue_type == 'Weather':
        ctype = 'Excavator Capacity'
        reduction_factor = np.random.uniform(0.50, 0.85)
        remark = np.random.choice(['Hujan deras', 'Badai', 'Material basah'])

    elif issue_type == 'Maintenance':
        ctype = np.random.choice(constraint_types)
        reduction_factor = np.random.uniform(0.20, 0.55)
        remark = np.random.choice(['Maintenance unit', 'Kerusakan', 'Perbaikan'])

    else:
        ctype = np.random.choice(constraint_types)
        reduction_factor = np.random.uniform(0.88, 1.00)
        remark = np.random.choice(['Normal', 'Operasi lancar'])

    constrained_value = max(500, int(base_cap * reduction_factor)) # Ensure a minimum of 500 ton/day

    production_list.append({
        'constraint_id': f"C{idx:05d}-{mine[-1]}",
        'mine_id': mine,
        'equipment_id': eq,
        'week_start': week,
        'constraint_type': ctype,
        'capacity_value': constrained_value,
        'unit': 'ton/day',
        'update_date': week + pd.Timedelta(days=np.random.randint(0, 7)),
        'remarks': remark
    })

production_constraints = pd.DataFrame(production_list)


In [ ]:
production_constraints.head()

,constraint_id,mine_id,equipment_id,week_start,constraint_type,capacity_value,unit,update_date,remarks
0,C00000-1,MINE_1,EQ037-1,2019-01-01,Excavator Capacity,500,ton/day,2019-01-05,Operasi lancar
1,C00001-1,MINE_1,EQ037-1,2019-01-08,Hauling Capacity,500,ton/day,2019-01-10,Akses terhambat
2,C00002-1,MINE_1,EQ033-1,2019-01-15,Hauling Capacity,500,ton/day,2019-01-20,Akses terhambat
3,C00003-1,MINE_1,EQ007-1,2019-01-22,Hauling Capacity,1386,ton/day,2019-01-25,Operasi lancar
4,C00004-1,MINE_1,EQ017-1,2019-02-01,Hauling Capacity,500,ton/day,2019-02-01,Operasi lancar


## Production constrain v2
tbh aku gak tau lagi

In [ ]:
# ================================================
# =====  PRODUCTION CONSTRAINTS (2-STATUS)  ======
# ================================================

np.random.seed(42)

production_list = []

# Merge effective_capacity dengan production_plan → supaya dapat status
eff_plan = effective_capacity.merge(
    production_plan[['plan_id', 'mine_id', 'week_start', 'status']],
    on=['plan_id', 'mine_id', 'week_start'],
    how='left'
)

# Severity hanya 2 level
status_severity = {
    "On Track": 0.95,
    "Below Target": 0.55   # jauh lebih rendah
}

# Remark per status
status_remarks = {
    "On Track": ["Operasi normal", "Alur produksi lancar"],
    "Below Target": ["Gangguan operasional", "Produksi tertunda", "Kondisi lapangan kurang optimal"]
}

constraint_types = ["Excavator Capacity", "Hauling Capacity", "Crusher Capacity"]

for idx, row in eff_plan.iterrows():

    mine = row['mine_id']
    week = row['week_start']
    eq = row['equipment_id']
    base_cap = row['effective_capacity_ton_day']
    status = row['status'] if row['status'] in ["On Track", "Below Target"] else "On Track"

    # Severity level
    sev = status_severity[status]

    # ============================
    # ISSUE TYPE PROBABILITY
    # ============================

    # On Track → mostly smooth
    if status == "On Track":
        issue_type = np.random.choice(
            ["None", "Road", "Weather", "Maintenance"],
            p=[0.60, 0.10, 0.20, 0.10]
        )

    # Below Target → mostly issues
    else:
        issue_type = np.random.choice(
            ["Road", "Weather", "Maintenance", "None"],
            p=[0.40, 0.35, 0.20, 0.05]
        )

    # ============================
    # ISSUE IMPACT RULES
    # ============================

    if issue_type == "Road":
        ctype = "Hauling Capacity"
        reduction_factor = np.random.uniform(0.25, 0.70) * sev
        remark = np.random.choice(["Akses terhambat", "Jalan rusak", "Kemacetan hauling"])

    elif issue_type == "Weather":
        ctype = "Excavator Capacity"
        reduction_factor = np.random.uniform(0.30, 0.75) * sev
        remark = np.random.choice(["Curah hujan tinggi", "Material basah", "Cuaca kurang mendukung"])

    elif issue_type == "Maintenance":
        ctype = np.random.choice(constraint_types)
        reduction_factor = np.random.uniform(0.20, 0.50) * sev
        remark = np.random.choice(["Maintenance unit", "Perbaikan alat", "Kerusakan mekanis"])

    else:  # No issue
        ctype = np.random.choice(constraint_types)
        reduction_factor = np.random.uniform(0.85, 1.00) * sev
        remark = np.random.choice(status_remarks[status])

    # ============================
    # FINAL CAPACITY VALUE
    # ============================

    constrained_value = max(500, int(base_cap * reduction_factor))

    production_list.append({
        'constraint_id': f"C{idx:05d}-{mine[-1]}",
        'mine_id': mine,
        'equipment_id': eq,
        'week_start': week,
        'constraint_type': ctype,
        'capacity_value': constrained_value,
        'unit': 'ton/day',
        'update_date': week + pd.Timedelta(days=np.random.randint(0, 7)),
        'remarks': remark
    })

production_constraints_2 = pd.DataFrame(production_list)

In [ ]:
production_constraints_2.head()

,constraint_id,mine_id,equipment_id,week_start,constraint_type,capacity_value,unit,update_date,remarks
0,C00000-1,MINE_1,EQ037-1,2019-01-01,Hauling Capacity,500,ton/day,2019-01-05,Kemacetan hauling
1,C00001-1,MINE_1,EQ037-1,2019-01-08,Excavator Capacity,500,ton/day,2019-01-10,Cuaca kurang mendukung
2,C00002-1,MINE_1,EQ033-1,2019-01-15,Excavator Capacity,500,ton/day,2019-01-20,Cuaca kurang mendukung
3,C00003-1,MINE_1,EQ007-1,2019-01-22,Hauling Capacity,500,ton/day,2019-01-27,Jalan rusak
4,C00004-1,MINE_1,EQ017-1,2019-02-01,Hauling Capacity,500,ton/day,2019-02-04,Akses terhambat


In [ ]:
production_constraints_2.tail()

,constraint_id,mine_id,equipment_id,week_start,constraint_type,capacity_value,unit,update_date,remarks
1195,C01195-1,MINE_1,EQ027-1,2025-12-22,Hauling Capacity,500,ton/day,2025-12-22,Kemacetan hauling
1196,C01196-4,MINE_4,EQ026-4,2025-12-22,Excavator Capacity,500,ton/day,2025-12-27,Perbaikan alat
1197,C01197-2,MINE_2,EQ049-2,2025-12-22,Hauling Capacity,500,ton/day,2025-12-23,Akses terhambat
1198,C01198-3,MINE_3,EQ039-3,2025-12-22,Excavator Capacity,500,ton/day,2025-12-22,Curah hujan tinggi
1199,C01199-5,MINE_5,EQ008-5,2025-12-22,Excavator Capacity,500,ton/day,2025-12-26,Material basah


# Shipping Schedule

Note dari rafi 17/11/25: ini aku gak edit sama sekali tapi aku takut dia ada error tolong cek yah ory

In [ ]:
# ================================
# ==== FIXED SHIPPING SCHEDULE ===
# ================================

np.random.seed(42)

# Define vessel and port mappings
vessels_by_region = {
    'Kalimantan Timur': ['MV Kalimantan Jaya', 'MV Borneo Star', 'MV Nusantara 01'],
    'Sumatera Selatan': ['MV Sriwijaya', 'MV Sumatra Raya'],
    'Kalimantan Selatan': ['MV Sungai Barito', 'MV Kapuas Indah']
}

port_mapping = {
    'Barito': ['Banjarmasin', 'Surabaya'],
    'Kutai Timur': ['Samarinda', 'Balikpapan'],
    'Batu Bara': ['Belawan', 'Tanjungbalai'],
    'Samarinda': ['Samarinda', 'Surabaya'],
    'Nunukan': ['Tarakan', 'Makassar']
}

ship_rows = []

for idx, row in production_plan.iterrows():

    mine = row['mine_id']
    week = row['week_start']
    region = mine_master[mine_master['mine_id']==mine]['region'].values[0]
    location = mine_master[mine_master['mine_id']==mine]['location'].values[0]

    # Vessel per region
    vessels = vessels_by_region.get(region, ['MV Nusantara 01']) # Default to Nusantara 01 if region not found
    vessel = np.random.choice(vessels)

    # Destination mapping per lokasi
    dest_options = port_mapping.get(location, ['Surabaya']) # Default to Surabaya if location not found
    dest_port = np.random.choice(dest_options)

    # ETD–ETA logic
    etd = week + pd.Timedelta(days=np.random.randint(0,6))

    if dest_port in ['Samarinda', 'Banjarmasin', 'Tarakan', 'Balikpapan']:
        transit_days = np.random.randint(1,4)
    elif dest_port in ['Surabaya', 'Makassar', 'Belawan', 'Tanjungbalai']:
        transit_days = np.random.randint(3,8)
    else:
        transit_days = np.random.randint(2,6)

    eta = etd + pd.Timedelta(days=transit_days)

    ship_rows.append({
        'shipment_id': f"SHP{idx:05d}-{mine[-1]}",   # konsisten dgn mine
        'mine_id': mine,
        'week_start': week,
        'vessel_name': vessel,
        'destination_port': dest_port,
        'coal_tonnage': np.random.randint(15000,40000),
        'etd': etd,
        'eta': eta,
        'status': np.random.choice(['On Schedule','Delayed','Pending'], p=[0.7,0.2,0.1])
    })

shipping_schedule = pd.DataFrame(ship_rows)

In [ ]:
shipping_schedule.head(10)

,shipment_id,mine_id,week_start,vessel_name,destination_port,coal_tonnage,etd,eta,status
0,SHP00000-1,MINE_1,2019-01-01,MV Nusantara 01,Surabaya,36575,2019-01-05,2019-01-10,On Schedule
1,SHP00001-1,MINE_1,2019-01-08,MV Nusantara 01,Surabaya,36962,2019-01-10,2019-01-15,Delayed
2,SHP00002-1,MINE_1,2019-01-15,MV Nusantara 01,Surabaya,38333,2019-01-19,2019-01-23,On Schedule
3,SHP00003-1,MINE_1,2019-01-22,MV Kalimantan Jaya,Banjarmasin,21396,2019-01-25,2019-01-27,On Schedule
4,SHP00004-1,MINE_1,2019-02-01,MV Kalimantan Jaya,Banjarmasin,39233,2019-02-03,2019-02-06,On Schedule
5,SHP00005-1,MINE_1,2019-02-08,MV Nusantara 01,Surabaya,16899,2019-02-13,2019-02-18,On Schedule
6,SHP00006-1,MINE_1,2019-02-15,MV Kalimantan Jaya,Banjarmasin,23838,2019-02-19,2019-02-22,On Schedule
7,SHP00007-1,MINE_1,2019-02-22,MV Nusantara 01,Surabaya,25555,2019-02-25,2019-02-28,On Schedule
8,SHP00008-1,MINE_1,2019-03-01,MV Borneo Star,Banjarmasin,38425,2019-03-02,2019-03-03,On Schedule
9,SHP00009-1,MINE_1,2019-03-08,MV Nusantara 01,Surabaya,32159,2019-03-11,2019-03-18,On Schedule


## Export Dataset

In [ ]:
print('{path}/dataset fixxx')

{path}/dataset fixxx


In [ ]:
mine_master.to_csv(f'{path}/dataset fixxx/mine_master.csv', index=False)
equipment_inventory.to_csv(f'{path}/dataset fixxx/equipment_inventory.csv', index=False)
weather_data.to_csv(f'{path}/dataset fixxx/weather_data.csv', index=False)
road_condition.to_csv(f'{path}/dataset fixxx/road_condition.csv', index=False)
production_plan.to_csv(f'{path}/dataset fixxx/production_plan.csv', index=False)
production_constraints.to_csv(f'{path}/dataset fixxx/production_constraints.csv', index=False)
production_constraints_2.to_csv(f'{path}/dataset fixxx/production_constrains_2.csv', index = False)
effective_capacity.to_csv(f'{path}/dataset fixxx/effective_capacity.csv', index=False)
effective_capacity_2.to_csv(f'{path}/dataset fixxx/effective_capacity_2.csv', index=False)
shipping_schedule.to_csv(f'{path}/dataset fixxx/shipping_schedule.csv', index = False)

In [ ]:
mine_master.head()

,mine_id,mine_name,location,region,status,start_date,remarks,start_year
0,MINE_1,Tambang Batubara Sungai Merah,Barito,Kalimantan Timur,Active,2018-12-31,Area utama produksi,2018
1,MINE_2,Tambang Batubara Tanjung Bara,Kutai Timur,Kalimantan Timur,Active,2019-12-31,Ekspansi,2019
2,MINE_3,Tambang Batubara Indrapura,Batu Bara,Sumatera Selatan,Active,2020-12-31,Cadangan batubara tinggi,2020
3,MINE_4,Tambang Batubara Mahakam,Samarinda,Kalimantan Timur,Active,2021-12-31,Dekat pelabuhan,2021
4,MINE_5,Tambang Batubara Sebuku,Nunukan,Kalimantan Selatan,Active,2022-12-31,Daerah baru,2022


In [ ]:
equipment_inventory.head()

,equipment_id,mine_id,equipment_type,brand,model,base_capacity_ton,status,last_maintenance,operator_id
0,EQ001-2,MINE_2,Loader,Volvo,L220H,14,Active,2025-09-01,OP001
1,EQ002-5,MINE_5,Dump Truck,Hitachi,EH4000AC-3,128,Maintenance,2025-09-04,OP002
2,EQ003-4,MINE_4,Dump Truck,BelAZ,75131,219,Active,2025-09-07,OP003
3,EQ004-2,MINE_2,Dump Truck,Volvo,A40G,220,Active,2025-09-10,OP004
4,EQ005-1,MINE_1,Dump Truck,Hitachi,EH4000AC-3,202,Maintenance,2025-09-13,OP005


In [ ]:
weather_data.head()

,weather_id,mine_id,date,rainfall_mm,temperature_c,wind_speed_kmh,humidity_pct,remark
0,W00000-1,MINE_1,2018-12-31,0.716591,27.02657,0.254774,87.217370,Mendung
1,W00001-1,MINE_1,2019-01-01,1.328278,26.07372,0.074442,91.921403,Hujan ringan
2,W00002-1,MINE_1,2019-01-02,0.813389,26.44854,0.207931,92.048375,Mendung
3,W00003-1,MINE_1,2019-01-03,0.158691,26.32665,0.157334,89.655613,Mendung
4,W00004-1,MINE_1,2019-01-04,0.036812,27.87400,0.313025,88.382719,Mendung


In [ ]:
road_condition.head()

,road_id,mine_id,segment_name,condition_level,accessibility_pct,last_inspection,remark
0,RD001-1,MINE_1,Segment 1,Good,97,2025-10-06,Lancar
1,RD002-1,MINE_1,Segment 2,Good,67,2025-10-27,Lancar
2,RD003-1,MINE_1,Segment 3,Good,80,2025-10-30,Perlu perbaikan
3,RD004-1,MINE_1,Segment 4,Fair,92,2025-10-05,Perlu perbaikan
4,RD005-1,MINE_1,Segment 5,Good,78,2025-10-04,Perlu perbaikan


In [ ]:
production_plan.head()

,plan_id,mine_id,week_start,planned_output_ton,actual_output_ton,target_variance_pct,status,updated_by
0,PLAN0001-1,MINE_1,2019-01-01,65131,47661.0,-26.82,Below Target,MPLNR02
1,PLAN0002-1,MINE_1,2019-01-08,76973,51518.0,-33.07,Below Target,MPLNR02
2,PLAN0003-1,MINE_1,2019-01-15,76973,50831.0,-33.96,Below Target,MPLNR02
3,PLAN0004-1,MINE_1,2019-01-22,76973,70184.0,-8.82,Below Target,MPLNR01
4,PLAN0005-1,MINE_1,2019-02-01,75986,51594.0,-32.10,Below Target,MPLNR01


In [ ]:
production_constraints.head()

,constraint_id,mine_id,equipment_id,week_start,constraint_type,capacity_value,unit,update_date,remarks
0,C0000-1,MINE_1,EQ023,2019-01-06,Excavator Capacity,500,ton/day,2019-01-10,Operasi lancar
1,C0001-1,MINE_1,EQ017,2019-01-13,Hauling Capacity,500,ton/day,2019-01-15,Akses terhambat
2,C0002-1,MINE_1,EQ043,2019-01-20,Hauling Capacity,500,ton/day,2019-01-24,Akses terhambat
3,C0003-1,MINE_1,EQ022,2019-01-27,Hauling Capacity,500,ton/day,2019-01-27,Maintenance unit
4,C0004-1,MINE_1,EQ033,2019-02-03,Hauling Capacity,500,ton/day,2019-02-03,Normal


In [ ]:
effective_capacity.head()

,effcap_id,plan_id,mine_id,equipment_id,week_start,distance_km,road_condition,weather_condition,availability_pct,effective_capacity_ton_day,remark
0,EFC0000-1,PLAN0001-1,MINE_1,EQ023,2019-01-06,3.658868,Good,Clear,94,66.0,Optimal
1,EFC0001-1,PLAN0002-1,MINE_1,EQ017,2019-01-13,4.814101,Good,Clear,82,32.0,Optimal
2,EFC0002-1,PLAN0003-1,MINE_1,EQ043,2019-01-20,10.424883,Good,Clear,98,39.0,Optimal
3,EFC0003-1,PLAN0004-1,MINE_1,EQ022,2019-01-27,3.679116,Good,Clear,80,50.0,Optimal
4,EFC0004-1,PLAN0005-1,MINE_1,EQ033,2019-02-03,15.742275,Good,Clear,83,31.0,Optimal


In [ ]:
shipping_schedule.head()

,shipment_id,mine_id,week_start,vessel_name,destination_port,coal_tonnage,etd,eta,status
0,SHP00000-1,MINE_1,2019-01-06,MV Nusantara 01,Samarinda,36575,2019-01-10,2019-01-13,On Schedule
1,SHP00001-1,MINE_1,2019-01-13,MV Nusantara 01,Samarinda,19426,2019-01-15,2019-01-18,On Schedule
2,SHP00002-1,MINE_1,2019-01-20,MV Nusantara 01,Samarinda,15769,2019-01-22,2019-01-24,Delayed
3,SHP00003-1,MINE_1,2019-01-27,MV Sinar Kalimantan,Samarinda,32568,2019-01-28,2019-01-29,On Schedule
4,SHP00004-1,MINE_1,2019-02-03,MV Sinar Kalimantan,Banjarmasin,23666,2019-02-06,2019-02-07,On Schedule
